In [13]:
!pip install sentence-transformers

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl.metadata (1.8 kB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinu

In [35]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from pathlib import Path
pdf_files = Path("/content/sample_data").glob("*.pdf")
final_docs=[]
for pdf in pdf_files:
  loader = PyPDFLoader(pdf)
  docs = loader.load()
  splitdocs = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
  splitted_docs = splitdocs.split_documents(docs)
  for i in splitted_docs:
    final_docs.append(i.page_content)

In [11]:
from langchain_community.retrievers import PineconeHybridSearchRetriever
import os
from dotenv import load_dotenv
from pinecone import Pinecone, ServerlessSpec
load_dotenv()
pinecone_api_key = os.environ["PINECONE_API_KEY"]
index_name = "rag-implementation-pinecone"
db = Pinecone(api_key=pinecone_api_key)

if index_name not in db.list_indexes().names():
  db.create_index(name=index_name, dimension=384, metric="dotproduct", spec=ServerlessSpec(cloud="aws", region="us-east-1"))

In [39]:
#connect to the pincone index
index = db.Index(index_name)

In [14]:
#creating vector embeddings
from langchain_community.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [15]:
#creating spare vector embeddings(TF-IDF)
from pinecone_text.sparse import BM25Encoder
bm25_encoder = BM25Encoder().default()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [40]:
#adding the documents in the pincone after applying dense vector embeddings and sparse vector embeddings
retriever = PineconeHybridSearchRetriever(embeddings=embeddings, sparse_encoder = bm25_encoder, index = index)
retriever.add_texts(final_docs)

  0%|          | 0/1 [00:00<?, ?it/s]

In [44]:
#llama3.1 Open-source LLM model
from langchain_groq import ChatGroq
groq_api_key = os.environ["GROQ_API_KEY"]
llm = ChatGroq(groq_api_key=groq_api_key, model="llama-3.1-8b-instant")

In [46]:
#prompt for the model
from langchain.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""Answer the question based on the context provided, make sure before giving answer <context>{context}</context> question :{input}""")

In [47]:
from langchain.chains.combine_documents import create_stuff_documents_chain
chain = create_stuff_documents_chain(llm, prompt)

In [54]:
while(True):
  query=input("Ask anything you want: ")
  if(query=="End"):
    print("Thank you so much, if you need anything ask me!")
    break
  else:
    result = retriever.invoke(query)
    response = chain.invoke({
        'context': result,
        'input': query
    })
    print(response)
    print()

Ask anything you want: how many documents are there ?
There are 2 documents.

Ask anything you want: can you brief me the documents ?
Here's a brief summary of the documents:

**Overview**: The documents appear to be a loan agreement between a borrower and a lender, with a guarantor addendum attached.

**Key Components**:

1. **Loan Agreement**: The main agreement outlines the terms and conditions of the loan, including the borrowed amount, interest rates, payment schedule, and collateral.
2. **Borrower's Signature**: The borrower (Steve Rogers/Tony Stark) has signed the agreement on two separate dates (09/08/2021 and 03/05/2023, respectively).
3. **Lender's Signature**: The lender (Hrudya/Tejesh) has signed the agreement on the same dates as the borrower.
4. **Guarantor Addendum**: The guarantor addendum outlines the terms and conditions of the loan as they apply to the guarantor, who is personally liable for the borrowed amount in the event of the borrower's default.
5. **Collateral*